## 27. 最尤推定（Maximum likelihood estimation）

### <font color = blue>**1.** </font> 最尤推定と確率分布のあてはめ

#### <font color = green> **1.1.** </font> 離散型確率分布 : ポアソン分布

$\displaystyle P(y)=\frac{{\bar y}^y \cdot e^{-\bar y}}{y!}\ \ (y=0,1,2,3,\cdots)$

In [ ]:
''' データの準備 : ポアソン分布からの乱数 '''

# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 乱数固定
np.random.seed(seed=10)

In [ ]:
# パラメータ λ = 2.4 で 1000個のデータを生成
poisson_values = np.random.poisson(lam=2.4, size=1000)

In [ ]:
## ヒストグラムで図示
# density=True で確率化する

sns.set()
p_y, bin_edges, patches = plt.hist(poisson_values , bins=11, range=[-0.5, 10.5], density=True)

In [ ]:
# y を出力（各ヒストの中心）
y_k = 0.5*(bin_edges[1:] + bin_edges[:-1])
print(y_k)

In [ ]:
# p(y) を出力
print(p_y)

In [ ]:
''' 対数尤度関数を偏微分してパラメータ推定 '''

# 代数計算する（微分や方程式を解く）ためのライブラリをインポート
import sympy 

In [ ]:
## 変数を定義
# λ および y という　「数式上の変数」　の記号を定義
sympy.var('λ y')

$\displaystyle P(y)=\frac{{\bar y}^y \cdot e^{-\bar y}}{y!}\ \ (y=0,1,2,3,\cdots)$

In [ ]:
## 尤度 p('パラメータ' | y) を定義
# sympy.factorial(y) -> y の階乗を返す関数
f = (λ**y) * sympy.exp(-λ) / sympy.factorial(y)

In [ ]:
f

In [ ]:
## 対数化
logf = sympy.log(f)

In [ ]:
logf

In [ ]:
## f を λ で偏微分して、式を展開（整理）
pdff = sympy.expand(sympy.diff(logf, λ))

In [ ]:
sympy.diff(logf, λ)

In [ ]:
pdff

In [ ]:
## 尤度の極値（今は極大値の想定）を与える変数（今は y を想定）を代数的に解く（今は解ける）

def L_sympy(f, var, values):
  # 尤度の初期化
  likelihood = 0

  # データの個数分繰り返す
  for i in np.arange(len(values)):
    # model output
    # print(values[i])

    # 変数（今は y を想定）に値を代入して合算
    likelihood += f.subs(var, values[i])

    # print(likelihood)

  # 尤度の合計を λ についての方程式として解く
  param = sympy.solve(likelihood, λ) 
  # print(param)

  return param

In [ ]:
## 計算実行
L_sympy(pdff, "y", poisson_values)

In [ ]:
# パラメータ推定量 ： [289/125] = 2.312

289/125

In [ ]:
''' 離散型確率分布の関数（確率質量関数） '''

def probability_poisson(y, λ):
  ## 「数式」　ではなく　「実際の数値」　の計算をしたいので...
  from scipy.special import factorial

  # λ : パラメータ
  # y : データY の頻度（y回発生する）

  # データY の確率 P(y | 'パラメータ') を返す
  return (λ**y) * np.exp(-λ) / factorial(y)

In [ ]:
''' 対数尤度関数 : 離散型 '''

def L_func(param, y):
  # 尤度の初期化
  likelihood = 0

  for i in np.arange(len(y)):
    ## model output
    p = probability_poisson(y[i], param)

    ## 尤度の符号を反転して合算
    likelihood += -np.log(p)

  ## 尤度の合計を返す
  return likelihood

In [ ]:
''' パラメータ推定 '''

'''
準Newton法（BFGS， L-BFGS法 ： 複雑な場合にメモリの節約などが可能）
'''

# 最大値や最小値など、とにかく与えた式・条件での最適解を推定してくれるライブラリ
from scipy import optimize

In [ ]:
# パラメータの初期値を設定
x0 = [2]

# 最適パラメータ検索の範囲(min, max)を設定
bound = [(0, None)]

In [ ]:
## 今は最小値（尤度ポテンシャルの底）を求めたい
# https://docs.scipy.org/doc/scipy/reference/optimize.minimize-lbfgsb.html

params_MLE = optimize.minimize(L_func,
                               x0,
                               args=(poisson_values),
                               method='l-bfgs-b',
                               bounds=bound)

In [ ]:
params_MLE

In [ ]:
# パラメータの最尤推定量
print('パラメータ ： {}'.format(params_MLE.x))

In [ ]:
## AIC（Akaike's Information Criterion : 赤池の情報量規準）
# -2*(最大対数尤度) + 2*(パラメータ数)
# 一番小さい値があてまりの良い確率分布

# パラメータ数
K = 1

# 今、そもそも L_func() 内で尤度の符号を反転して合算しているので...
print('AIC ： {}'.format(2*params_MLE.fun + 2*K))

In [ ]:
''' curve_fitでとりあえずパラメータ計算する手法もあり '''

from scipy.optimize import curve_fit

parameters, cov_matrix = curve_fit(f=probability_poisson,
                                   xdata=y_k,
                                   ydata=p_y
                                   ) 

print("パラメータ : ", parameters, "\t 共分散 : ", cov_matrix)

In [ ]:
''' 計算した最大尤度における当てはまり具合を図示してみる '''

acc_mle = probability_poisson(y_k, params_MLE.x)

plt.hist(poisson_values, bins=11, range=[-0.5, 10.5], density=True)
plt.plot(y_k, acc_mle)
plt.show()

#### <font color = green> **1.2.** </font> 連続型確率分布 : 正規分布

$\displaystyle P(y)=\frac{1}{\sqrt{2\pi\sigma^2}}\ exp{\left(-\dfrac{(y_i - \mu)^2}{2\sigma^2}\right)}$

In [ ]:
''' データの準備 : 正規分布からの乱数 '''

## ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 乱数固定
np.random.seed(seed=10)

In [ ]:
# 平均0, 標準偏差1 の正規分布から1000個のデータを生成 
norm_values = np.random.normal(0, 1, size=1000)

In [ ]:
## ヒストグラムで図示

sns.set()
plt.hist(norm_values,
         bins=50,
         range=[-5, 5], # 幅10に50本 -> 0.2刻みで集計
         #density=True # 確率化せずそのままの頻度で表示
         )

plt.show()
#bin_edges

In [ ]:
''' 対数尤度関数を偏微分してパラメータ推定 '''

# 代数計算する（微分や方程式を解く）ためのライブラリをインポート 
import sympy

In [ ]:
# 変数を定義（ σ**2 = v とおく）
sympy.var('μ v y')

$\displaystyle P(y)=\frac{1}{\sqrt{2\pi\sigma^2}}\ exp{\left(-\dfrac{(y_i - \mu)^2}{2\sigma^2}\right)}$

In [ ]:
# 尤度 p('パラメータ' | x) を定義
fe = (1 / sympy.sqrt(2 * sympy.pi * v)) * sympy.exp(-(y - μ)**2 / (2 * v))

In [ ]:
# 対数化
logf = sympy.log(fe)

In [ ]:
## fを偏微分して、式を展開
# μについて偏微分
pdff1 = sympy.expand(sympy.diff(logf, μ)) 

In [ ]:
pdff1

In [ ]:
# vについて偏微分
pdff2 = sympy.expand(sympy.diff(logf, v))

In [ ]:
pdff2

$\mu$ について偏微分した $\verb |pdff1|=\dfrac{y}{v} - \dfrac{\mu}{v}$


$v$ について偏微分した $\verb |pdff2|=-\dfrac{1}{2v} + \dfrac{y^2}{2v^2} - \dfrac{y\mu}{v^2} + \dfrac{\mu^2}{2v^2}$


$\sum{\verb |pdff1|} = \sum{\verb |pdff2|} = 0$ になる $\mu, v$ を求める

In [ ]:
## 尤度を計算して方程式を解く関数
def L_sympy(fmu, fs, var, values):
  # 尤度の初期化
  likelihood_mu = 0
  likelihood_s = 0

  for i in np.arange(len(values)):
    # 各データの尤度を計算して合算していく
    likelihood_mu += fmu.subs(var,values[i])
    likelihood_s += fs.subs(var,values[i])
  
  # 方程式を解く
  param = sympy.solve([likelihood_mu, likelihood_s]) 
  return param

In [ ]:
# 計算実行
parameters = L_sympy(pdff1, pdff2, "y", norm_values)

In [ ]:
# σ**2 = v とおいてここまで計算したので、v を　σ に変換する
parameters[0]["σ"] = sympy.sqrt(parameters[0][v])

In [ ]:
# 計算結果表示
parameters

In [ ]:
## 以下、1.1. と同様の流れ

''' 確率密度関数 '''
def probability_function(y, param):
  from scipy.special import factorial
  # param[0] : sigma
  # param[1] : mu
  # y : データY
  # return : データY の確率密度 P(y | 'パラメータ')
  return (1/np.sqrt(2 * np.pi * param[1]**2)) * np.exp(-0.5 * (y - param[0])**2 /param[1]**2)

$\displaystyle P(y)=\frac{1}{\sqrt{2\pi\sigma^2}}\ exp{\left(\dfrac{-1}{2}\cdot\dfrac{(y_i - \mu)^2}{\sigma^2}\right)}$

In [ ]:
''' 対数尤度関数（連続） '''
def L_func_c(param, y):
  # 尤度の初期化
  likelihood = 0
  for i in np.arange(len(y)):
    # データごとに確率密度を算出
    p = probability_function(y[i], param)
    # 尤度の符号を反転させて合算
    likelihood += -np.log(p)
  return likelihood

In [ ]:
''' パラメータ推定 '''
'''
準Newton法（BFGS， L-BFGS法 ： 複雑な場合にメモリの節約などが可能）
'''

from scipy import optimize

In [ ]:
# パラメータの初期値を設定
x0 = [0, 0.1]

In [ ]:
#　最適パラメータ検索の範囲(min,　max)を設定
bound = [(-100, 100), (0, None)]

In [ ]:
## 最小値（尤度ポテンシャルの底）を求める
params_MLE = optimize.minimize(L_func_c,
                               x0,
                               args=(norm_values),
                               method='l-bfgs-b',
                               bounds=bound
                               )

In [ ]:
# 最尤推定パラメータ
print('μ : {},  σ ： {}'.format(params_MLE.x[0], params_MLE.x[1]))

In [ ]:
## AIC

# パラメータ数
k = 2

print('AIC ： {}'.format(2*params_MLE.fun + 2*k))

In [ ]:
## scipy.stats.fit でパラメータ推定すると...

from scipy.stats import norm
fit_parameter = norm.fit(norm_values)
fit_parameter

In [ ]:
## 推定結果の当てはまり具合を図示する
acc_mle = probability_function(np.sort(norm_values), params_MLE.x)

# 今度は density=True で確率密度化した状態を表示
plt.hist(norm_values, bins=50, range=[-5, 5], density=True)
plt.plot(np.sort(norm_values), acc_mle)
plt.show()

### <font color = blue>**2.** </font> pdf教材のコードより

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import math
import time
import random

In [ ]:
sample = 100  # 10000だと計算に時間がかかりすぎるので...
M = 0.5
S_max, S_min, m_step, s_step = 1.3, 0.7, 0.01, 0.01

In [ ]:
data = np.random.randn(sample)

In [ ]:
Sigma = int((S_max - S_min) / s_step)
Mu = int(2 * M / m_step)

In [ ]:
n = np.zeros((Mu, Sigma))
# print("n : {}".format(n))

In [ ]:
t1 = time.time()

for s in range(Sigma):
  for m in range(Mu):
    nn = 0.0
    nf = [norm.pdf(data[i], loc=(m_step * m - M), scale=(s_step * s + S_min)) for i in range(sample)]
    for i in range(sample):
      if nf[i] != 0:
        nn = nn + math.log2(nf[i])
    n[m][s] = nn

t2 = time.time()
print("sample = {}, total time : {}".format(sample, (t2 - t1)))
### 1分程度で完了するはず

print("n : {}".format(n))

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
mu = [(m*m_step - M) for m in range(Mu)]
sigma = [(s*s_step + S_min) for s in range(Sigma)]
s, m = np.meshgrid(sigma, mu)

In [ ]:
plt.figure(figsize=(15,15))
plt.contourf(s, m, n,
             levels=1000)
plt.contour(s, m, n, cmap='RdYlGn',
            levels=400)
plt.show()

### <font color = blue>**3.** </font> 確率的勾配降下法（SGD : Stochastic Gradient Descent）

$\downarrow \downarrow$ scikit-learnリファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/sgd.html

#### <font color = green> **3.1.** </font> 自作して計算速度を比較（pdf教材のコードより）

In [ ]:
def grad_gaussian_x(x, mu, sigma):
  return (x - mu)/sigma

def grad_gaussian_y(x, mu, sigma):
  return -1/(2 * sigma) + (x - mu) * (x - mu)/(2 * sigma * sigma)

def grad_likelihood_x(data, sample, mu, sigma):
  return np.sum(np.array([grad_gaussian_x(data[i], mu, sigma) for i in range(sample)]))

def grad_likelihood_y(data, sample, mu, sigma):
  return np.sum(np.array([grad_gaussian_y(data[i], mu, sigma) for i in range(sample)]))

In [ ]:
## 勾配降下法をフルに計算してみると...?

sample = 1000 # pdf教材だと10000、計算に4分くらいかかる
data = np.random.randn(sample)

epsilon, mu, sigma = 0.00001, 0.5, 0.5
runtime = 10000

t1 = time.time()

for i in range(runtime):
  mu = mu + epsilon * grad_likelihood_x(data, sample, mu, sigma)
  sigma = sigma + epsilon * grad_likelihood_y(data, sample, mu, sigma)

t2 = time.time()
print("total time : {}".format(t2 - t1))

print("mu : {} \t sigma : {}".format(mu, sigma))

In [ ]:
## 同じパラメータ設定で確率的勾配降下法（SGD）を計算してみる

t1 = time.time()

for i in range(runtime):
  rr = random.randint(0, sample-1)
  mu = mu + epsilon * grad_gaussian_x(data[rr], mu, sigma)
  sigma = sigma + epsilon * grad_gaussian_y(data[rr], mu, sigma)

t2 = time.time()
print("total time : {}".format(t2 - t1))

print("mu : {} \t sigma : {}".format(mu, sigma))

#### <font color=red> task : </font> 以下のパラメータに対しグリッドサーチ的探索をする
- sample
- epsilon
- runtime

確率的にする/しないことにより、勾配降下法の計算速度にどれくらい違いが出るかを実感してほしい

In [ ]:
### EXAMPLE ###
## 勾配降下法の計算を関数化
def myOwnDG(sample=1000, epsilon=0.00001, mu=0.5, sigma=0.5, runtime=10000):
  data = np.random.randn(sample)
  mu_tmp = mu
  sigma_tmp = sigma
  t1 = time.time()

  for i in range(runtime):
    mu_tmp = mu_tmp + epsilon * grad_likelihood_x(data, sample, mu_tmp, sigma_tmp)
    sigma_tmp = sigma_tmp + epsilon * grad_likelihood_y(data, sample, mu_tmp, sigma_tmp)

  t2 = time.time()
  print("total time : {}".format(t2 - t1))

  print("mu : {} \t sigma : {}".format(mu, sigma))

In [ ]:
## 確率的勾配効果法を関数化
def myOwnSDG(sample=1000, epsilon=0.00001, mu=0.5, sigma=0.5, runtime=10000):
  data = np.random.randn(sample)
  mu_tmp = mu
  sigma_tmp = sigma
  t1 = time.time()

  for i in range(runtime):
    rr = random.randint(0, sample-1)
    mu_tmp = mu_tmp + epsilon * grad_gaussian_x(data[rr], mu_tmp, sigma_tmp)
    sigma_tmp = sigma_tmp + epsilon * grad_gaussian_y(data[rr], mu_tmp, sigma_tmp)

  t2 = time.time()
  print("total time : {}".format(t2 - t1))

  print("mu : {} \t sigma : {}".format(mu_tmp, sigma_tmp))

In [ ]:
## sample を比較探索
sample_list  = [1e2,
                1e3,
                1e4,
                1e5,
                1e6]

for i in sample_list:
  print("\n runtime = {}".format(int(i)))
  myOwnDG(sample=int(i))
  myOwnSDG(sample=int(i))

In [ ]:
## epsilon を比較探索
epsilon_list = [0.00005,
                0.0001,
                0.001,
                0.01]

for i in epsilon_list:
  print("\n epsilon = {}".format(i))
  myOwnDG(epsilon=i)
  myOwnSDG(epsilon=i)

In [ ]:
## runtime を比較探索
runtime_list = [1e3,
                1e4,
                1e5,
                1e6,
                1e7]

for i in runtime_list:
  print("\n runtime = {}".format(int(i)))
  myOwnDG(runtime=int(i))
  myOwnSDG(runtime=int(i))

#### <font color = green> **3.2.** </font> irisデータを使った公式サンプルコード

ライブラリ : sklearn.linear_model.SGDClassifier

$\downarrow \downarrow$ 公式リファレンス $\downarrow \downarrow$\
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html?highlight=sgd

In [ ]:
## Plot multi-class SGD on the iris dataset
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.linear_model import SGDClassifier

In [ ]:
# import some data to play with
iris = datasets.load_iris()

# we only take the first two features. We could
# avoid this ugly slicing by using a two-dim dataset
X = iris.data[:, :2]
y = iris.target
colors = "bry"

In [ ]:
# shuffle
idx = np.arange(X.shape[0])
np.random.seed(13)
np.random.shuffle(idx)
X = X[idx]
y = y[idx]

In [ ]:
# standardize
mean = X.mean(axis=0)
std = X.std(axis=0)
X = (X - mean) / std

In [ ]:
h = .02  # step size in the mesh

clf = SGDClassifier(alpha=0.001, max_iter=100).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

In [ ]:
# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(figsize=(8,8))
cs = plt.contourf(xx, yy, Z, cmap=plt.cm.Wistia)
plt.axis('tight')

# Plot also the training points
for i, color in zip(clf.classes_, colors):
  idx = np.where(y == i)
  plt.scatter(X[idx, 0], X[idx, 1], c=color, label=iris.target_names[i],
              cmap=plt.cm.Paired, edgecolor='black', s=20)
plt.title("Decision surface of multi-class SGD")
plt.axis('tight')

# Plot the three one-against-all classifiers
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()
coef = clf.coef_
intercept = clf.intercept_


def plot_hyperplane(c, color):
  def line(x0):
    return (-(x0 * coef[c, 0]) - intercept[c]) / coef[c, 1]
  
  plt.plot([xmin, xmax], [line(xmin), line(xmax)],
           ls="--", color=color)


for i, color in zip(clf.classes_, colors):
  plot_hyperplane(i, color)

plt.legend()
plt.show()

#### <font color=red> task : </font> 上記の公式サンプルコードに対して以下の2点を行う（コードの読解）
- 最終実行結果（2×2のグラフの出力＆その内容）が変わらない範囲で可能な限りブロックごとに実行セルを分割する
- セル毎/行毎に、何をしているか / 何をしたいか の説明をコメントで加筆する

#### <font color = green> **3.3.** </font> ワインのデータをSGDで線形分類

In [ ]:
## ライブラリのインポート

# numpyという行列などを扱うライブラリを利用
import numpy as np 

# pandasというデータ分析ライブラリを利用
import pandas as pd 

# プロット用のライブラリを利用
import matplotlib.pyplot as plt 

# 機械学習用のライブラリを利用
from sklearn import linear_model, metrics, preprocessing, model_selection 

# 学習結果をプロットする外部ライブラリを利用
from mlxtend.plotting import plot_decision_regions 

In [ ]:
## データセットを読み込む
df_wine_all = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)

# 生データはこんな感じ
pd.DataFrame(df_wine_all)

In [ ]:
## 品種(0列、1～3)と色（10列）とプロリンの量(13列)を使用する
df_wine = df_wine_all[[0, 10, 13]]
df_wine.columns = [u'class', u'color', u'proline']

# この行を実行するとデータが見れる
pd.DataFrame(df_wine)  

In [ ]:
## プロットしてみる
 
x = df_wine["color"]
y = df_wine["proline"]
z = df_wine["class"]-1
plt.scatter(x, y, c=z)
plt.show

In [ ]:
## データの整形
# 説明変数を抜き出して標準化
X = df_wine[["color", "proline"]]
sc = preprocessing.StandardScaler()
sc.fit(X)
X_std = sc.transform(X)

In [ ]:
## 機械学習で分類する
clf_result = linear_model.SGDClassifier(loss="hinge")

In [ ]:
## K分割交差検証（cross validation）で性能を評価する
scores = model_selection.cross_val_score(clf_result, X_std, z, cv=10)
print("\t平均正解率 \t= ", scores.mean())
print("正解率の標準偏差 \t= ", scores.std())

In [ ]:
## トレーニングデータとテストデータに分けて実行してみる
X_train, X_test, train_label, test_label = model_selection.train_test_split(X_std,z, test_size=0.1, random_state=0)
clf_result.fit(X_train, train_label)

In [ ]:
# 正答率を求める
pre = clf_result.predict(X_test)
ac_score = metrics.accuracy_score(test_label, pre)
print("正答率 = ", ac_score)

In [ ]:
# plotする
X_train_plot = np.vstack(X_train)
train_label_plot = np.hstack(train_label)
X_test_plot = np.vstack(X_test)
test_label_plot = np.hstack(test_label)

# 学習データをプロット
#plot_decision_regions(X_train_plot, train_label_plot, clf=clf_result, res=0.01) 

# テストデータをプロット
plot_decision_regions(X_test_plot, test_label_plot, clf=clf_result, res=0.01, legend=2)

## Matplotlib 3.3.0 でいろいろ変わった関係で mlxtend.plotting.plot_decision_regions にWarningが出る ##

In [ ]:
## 任意のデータに対する識別結果を見てみる
predicted_label = clf_result.predict([[1,-1]])
print("このテストデータのラベル = ", predicted_label)

In [ ]:
## 識別平面の式を手に入れる
# coef[0] * x + coef[1] * y + intercept = 0
print(clf_result.intercept_)
print(clf_result.coef_ )

## 28. ロジスティック回帰（Logistic regression）

### <font color = blue>**1.** </font> １変数２値分類

In [ ]:
# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## 青森県青森市の気象観測データ(2017年11月～2018年1月)
# Rain_or_Snow

## データ生成（手動入力）
# 最低気温
X = np.array([-6.5, -6.2, -6.1, -5.9, -5.3, -5.2, -5.2, -4.8,
              -4.5, -4.1, -4.1, -3.9, -3.9, -3.7, -3.6, -3.6,
              -3.5, -3.5, -3.4, -3.0, -2.9, -2.8, -2.7, -2.7,
              -2.6, -2.5, -2.5, -2.4, -2.3, -2.0, -2.0, -1.9,
              -1.8, -1.6, -1.3, -1.2, -1.2, -1.0, -1.0, -1.0,
              -0.9, -0.9, -0.9, -0.9, -0.8, -0.5,  0.0,  0.1,
               0.1,  0.2,  0.2,  0.3,  0.4,  0.5,  1.1,  2.1,
               4.3,  4.3,  4.9,  5.5,  6.5,  7.1,  7.5,  7.8])

# 降雪／降雨現象(雪=0, 雨=1)
T = np.array([0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0,
              0, 0, 0, 0, 0, 0, 0, 0,
              0, 1, 0, 0, 0, 0, 0, 0,
              0, 0, 1, 1, 1, 0, 0, 0,
              0, 0, 0, 0, 0, 1, 1, 0,
              0, 1, 1, 1, 0, 0, 0, 1,
              1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# Tが1となるXの要素で配列を生成し、その中から最小要素を抽出
# （雨が降ったときの最低気温）
Z = X[T == 1]
x_1 = np.min(Z)

In [ ]:
# Tが0となるXの要素で配列を生成し、その中から最大要素を抽出
# （雪が降ったときの最高気温）
Z = X[T == 0]
x_2 = np.max(Z)

In [ ]:
## 図示

# 何度も使うので関数化しておく
def GraphShow():
  # 描画設定初期化
  sns.reset_defaults()

  # FigureとAxesの設定
  fig, ax = plt.subplots(figsize=(8, 5))
  ax.grid()
  ax.set_xlim(-8, 8)
  ax.set_ylim(-0.5, 1.5)
  ax.set_yticks([0, 1])
  ax.set_xlabel("Temperature [${}^\circ$C]", fontsize = 14)

  # 雨は赤色、雪は青色の点で表示
  ax.scatter(X[T == 0], T[T == 0], color = "blue", s = 16, label = "snow", zorder = 2)
  ax.scatter(X[T == 1], T[T == 1], color = "red", s = 16, label = "rain", zorder = 2)

  # Tが0と1の両方を取り得る範囲を塗り潰す
  ax.axvspan(x_1, x_2, color = "orange", alpha = 0.4, zorder = 1)

  # x軸の目盛を設定
  xt = range(-8, 9, 4)
  xt = list(xt) + [x_1, x_2]
  ax.set_xticks(xt)

  return ax

In [ ]:
ax = GraphShow()
ax.legend()
plt.show()

#### <font color = green> **1.1.** </font> シグモイド関数でフィッティング

$\displaystyle \sigma(x) = \frac{1}{1+e^{-ax}} \ \ (a>0)$

In [ ]:
# シグモイド関数
def sigmoid(x, a):
  f = 1 / (1 + np.exp(-a*x))
  return f

In [ ]:
## 適当な a でシグモイド曲線を描画

a_list = [0.5, 1, 2]
color_list = ["green", "red", "blue"]

ax = GraphShow()

x = np.linspace(-8, 8, 257)
for a, color in zip(a_list, color_list):
  S = sigmoid(x, a)
  ax.plot(x, S, linestyle='dashed', color = color, label="a = {}".format(a))

ax.legend()
plt.show()

In [ ]:
### a を自由に変更して描画してみましょう ###
a_list = [ , , , , , , ]
color_list = ["green", "red", "blue", 'cyan', 'magenta', 'yellow', 'black']


ax = GraphShow()

x = np.linspace(-8, 8, 257)
for a, color in zip(a_list, color_list):
  S = sigmoid(x, a)
  ax.plot(x, S, linestyle='dashed', color = color, label="a = {}".format(a))

ax.legend()
plt.show()

#### <font color = green> **1.2.** </font> ロジスティック関数でフィッティング

$\displaystyle L(x) = \sigma(ax+b) = \frac{1}{1+\exp\{-(ax+b)\}}$

In [ ]:
# ロジスティック関数
def logistic(x, q):
  f = 1 / (1 + np.exp(-(q[0] * x + q[1])))
  return f

In [ ]:
# 適当な q=(a, b) でロジスティック曲線を描画
q_list = [(1, -3), (1, 0), (1, 3)]
color_list = ["green", "red", "blue", 'cyan', 'magenta', 'yellow', 'black']

ax = GraphShow()

x = np.linspace(-8, 8, 257)
for q, color in zip(q_list, color_list):
  S = logistic(x, q)
  ax.plot(x, S, linestyle='dashed', color = color, label="(a, b) = ({}, {})".format(q[0], q[1]))

ax.legend()
plt.show()

In [ ]:
### q=(a, b) を自由に変更して描画してみましょう ###
q_list = [( , ), ( , ), ( , ), ( , ), ( , ), ( , ), ( , )]
color_list = ["green", "red", "blue", 'cyan', 'magenta', 'yellow', 'black']

ax = GraphShow()

x = np.linspace(-8, 8, 257)
for q, color in zip(q_list, color_list):
  S = logistic(x, q)
  ax.plot(x, S, linestyle='dashed', color = color, label="(a, b) = ({}, {})".format(q[0], q[1]))

ax.legend()
plt.show()


#### <font color = green> **1.3.** </font> フィッティング精度の評価

In [ ]:
## 平均交差エントロピー誤差関数
def cee_logistic(q, x, t):
  f = logistic(x, q)
  c = - np.sum((t * np.log(f) + (1 - t) * np.log(1 - f)))
  return c / len(f)

In [ ]:
## 平均交差エントロピー誤差関数の勾配
def grad_cee_logistic(q, x, t):
  f = logistic(x, q)
  grad = np.zeros(2)
  grad[0] = np.sum((f - t) * x)
  grad[1] = np.sum(f - t)
  return grad / len(f)

In [ ]:
## 平均交差エントロピー誤差関数が最小値をとるパラメータを探索する
# ライブラリのインポート
from scipy.optimize import minimize

In [ ]:
# パラメータの初期値
q0 = [2, -2]

In [ ]:
## 最尤パラメータの探索実行
fit = minimize(cee_logistic,
               q0,
               args=(X, T),
               jac=grad_cee_logistic,
               method="CG")

In [ ]:
# 最尤パラメータを取得
q = fit.x
print("(a, b) = ({0:.3f}, {1:.3f})".format(q[0], q[1]))

In [ ]:
## 最適化されたパラメータでロジスティック曲線を描画
# 決定境界も追加

ax = GraphShow()

# 最尤ロジスティック曲線
x = np.linspace(-8, 8, 257)
L = logistic(x, q)
ax.plot(x, L, color = "green")

# Tが1（雨）となる確率が0.5を超える要素のインデックスを取得
L2 = np.where(L > 0.5)

# L2の最小要素
i = np.min(L2)

# 決定境界
b = (x[i] + x[i -1]) / 2
print("決定境界 x = {0:.3f}".format(b))

# 決定境界線
ax.plot([b, b], [-2, 2], color = "black",
        linestyle = "dashed", label = "Decision Boundary")

ax.legend()
plt.show()

### <font color = blue>**2.** </font> ２変数２値分類

In [ ]:
# ライブラリのインポート
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from scipy.optimize import minimize

In [ ]:
## 北海道室蘭市における気象観測データ（2017年11月1日～2018年1月31日）
# bool Snowfall

## データ生成（手動入力）
# 最低気温[℃]
X1 = np.array([-10.1, -9.9, -7.8, -7.4, -6.9, -6.8, -6.5, -6.4, -5.5,
               -5.4, -5.2, -5.0, -4.9, -4.9, -4.7, -4.7, -4.5, -4.5,
               -4.3, -4.3, -4.3, -4.1, -4.1, -4.1, -4.0, -3.8, -3.8,
               -3.6, -3.5, -3.4, -3.4, -3.3, -3.2, -3.1, -3.1, -3.0,
               -3.0, -2.9, -2.9, -2.9, -2.6, -2.5, -2.4, -2.3, -2.3,
               -2.2, -2.1, -2.1, -2.1, -2.0, -1.8, -1.6, -1.6, -1.5,
               -1.5, -1.4, -1.3, -1.0, -0.8, -0.4, -0.3, -0.2, -0.2,
               -0.1,  0.0,  0.3,  0.6,  0.7,  0.7,  0.7,  0.9,  1.0,
               1.1,  1.3,  1.7,  2.0,  2.5,  2.8,  3.6,  3.9,  4.0,
               4.1,  4.3,  4.4,  4.7,  7.4,  7.9,  9.7,  9.8,  11.1])

# 平均湿度[%]
Y1 = np.array([74, 67, 60, 63, 65, 61, 72, 70, 73,
               74, 69, 74, 67, 81, 72, 68, 74, 66,
               73, 70, 71, 65, 58, 71, 76, 66, 81,
               67, 85, 62, 70, 67, 70, 69, 75, 74,
               71, 79, 79, 69, 68, 77, 75, 79, 68,
               72, 72, 72, 75, 58, 83, 64, 73, 74,
               68, 67, 78, 67, 69, 79, 84, 67, 72,
               69, 77, 78, 69, 69, 74, 66, 73, 74,
               82, 61, 74, 70, 71, 81, 89, 69, 63,
               63, 63, 77, 58, 72, 87, 83, 80, 75])

Y2 = Y1 / 100

# 降雪があった場合=0 / 降雪がなかった場合=1
T1 = np.array([0, 0, 1, 1, 0, 1, 1, 0, 1,
               0, 0, 0, 1, 0, 0, 1, 0, 1,
               0, 0, 1, 0, 1, 1, 0, 1, 1,
               1, 0, 1, 1, 1, 0, 0, 0, 0,
               0, 0, 0, 1, 1, 1, 0, 0, 0,
               1, 0, 1, 0, 1, 0, 1, 1, 0,
               0, 0, 1, 0, 1, 0, 1, 1, 1,
               1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1, 1, 1, 1, 1, 1, 1,
               1, 1, 1, 1, 1, 1, 1, 1, 1])

In [ ]:
# 降雪現象が観測された日の最低気温と平均湿度
X_0 = X1[T1 == 0]
Y_0 = Y1[T1 == 0]
Y2_0 = Y2[T1 == 0]

In [ ]:
# 降雪現象が観測されなかった日の最低気温と平均湿度
X_1 = X1[T1 == 1]
Y_1 = Y1[T1 == 1]
Y2_1 = Y2[T1 == 1]

In [ ]:
## 図示

# 一応関数化しておく
def GraphShow2():
  # 描画設定初期化
  sns.reset_defaults()

  # FigureとAxesの設定
  fig, ax = plt.subplots(figsize=(8, 5))
  ax.grid()
  ax.set_xlim(-10, 10)
  ax.set_ylim(50, 90)
  ax.set_xlabel("Temperature [${}^\circ$C]", fontsize = 14)
  ax.set_ylabel("Daily mean humidity [%]", fontsize = 14)
 
  # 降雪現象が観測された日のデータを青色で表示
  ax.scatter(X_0, Y_0, color="blue", s=16, label="snow", zorder=2)

  # 降雪現象が観測されなかった日のデータを赤色で表示
  ax.scatter(X_1, Y_1, color="red", s=16, label="rain", zorder=2)

  return ax

In [ ]:
ax = GraphShow2()
ax.legend()
plt.show()

#### <font color = green> **2.1.** </font> 2変数ロジスティック関数でフィッティング

In [ ]:
# 2変数ロジスティック関数
def logistic_2(x, y, q):
  f = 1 / (1 + np.exp(-(q[0] * x + q[1] * y + q[2])))
  return f

In [ ]:
# ２変数版 平均交差エントロピー誤差関数
def cee_logistic_2(q, x, y, t):
  f = logistic_2(x, y, q)
  c = - np.sum((t * np.log(f) + (1 - t) * np.log(1 - f)))
  return c / len(x)

In [ ]:
# ２変数版 平均交差エントロピー誤差関数の勾配
def grad_cee_logistic_2(q, x, y, t):
  f = logistic_2(x, y, q)
  grad = np.zeros(3)
  grad[0] = np.sum((f - t) * x)
  grad[1] = np.sum((f - t) * y)
  grad[2] = np.sum(f - t)
  return grad / len(x)

In [ ]:
# パラメータの初期値
q0 = [2, -5, 5]

In [ ]:
# 平均交差エントロピー誤差関数が最小値をとるパラメータを探索
fit2 = minimize(cee_logistic_2,
                q0,
                args=(X1, Y2, T1),
                jac=grad_cee_logistic_2,
                method="CG")

In [ ]:
# パラメータを取得
q = fit2.x
print("(a, b, c) = ({0:.3f}, {1:.3f}, {2:.3f})".format(q[0], q[1], q[2]))

In [ ]:
## 3Dで図示
# 格子点生成関数を定義
def mesh_2d(xr, yr, n):
  x = np.linspace(xr[0], xr[1], n)
  y = np.linspace(yr[0], yr[1], n)
  X, Y = np.meshgrid(x, y)
  return X, Y

In [ ]:
# 格子点を作成
XX, YY = mesh_2d([-10, 10], [0.5, 1.0], 129)

In [ ]:
# 高度を計算
TT = logistic_2(XX, YY, q)

In [ ]:
# FigureとAxes
fig = plt.figure(figsize = (8, 6))
ax = fig.add_subplot(111, projection="3d")
ax.set_title("Logistic function of 2 Variables", fontsize=16, position=(0.5, 1.1))
ax.set_xlabel("x", size=12)
ax.set_ylabel("y", size=12)
ax.set_zlabel("P (t=1)", size=12)
ax.set_zlim([0, 1])

# 視点の設定
ax.view_init(45, 135)

# 曲面を描画
ax.plot_surface(XX, YY, TT, color = "cyan")

plt.show()

In [ ]:
## 平面上に決定境界線を引く

# 等高線描画データ
xr = [-10, 10]
yr = [0.5, 1.0]
n = 257
XX, YY = mesh_2d(xr, yr, n)
TT = logistic_2(XX, YY, q)

In [ ]:
# Figureと3DAxesの設定
fig = plt.figure(figsize = (8, 6))
ax = fig.add_subplot(111)
ax.set_xlabel("Temperature [${}^\circ$C]", fontsize=14)
ax.set_ylabel("Daily mean humidity [%]", fontsize=14)
ax.set_xlim([xr[0], xr[1]])
ax.set_ylim([yr[0], yr[1]])

# 等高線（=決定境界線）をプロット
ct = ax.contour(XX, YY, TT, colors="black", levels=[0.5], zorder=1)
ax.clabel(ct, fmt="%0.1f", fontsize=10)

# 降雪現象が観測された日のデータを青色で表示
ax.scatter(X_0, Y2_0, color="blue", s=16, label="snow", zorder=2)

# 降雪現象が観測されなかった日のデータを赤色で表示
ax.scatter(X_1, Y2_1, color="red", s=16, label="rain", zorder=2)

ax.set_yticklabels(np.arange(50, 110, 10))
ax.legend()

### <font color = blue>**3.** </font> ソフトマックス関数の導入（pdf教材のコードより）

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import random
import time
import matplotlib.pyplot as plt

In [ ]:
iris = load_iris()

In [ ]:
iris_target = iris.target

In [ ]:
iris_data = iris.data[:, 2:4]  ###

In [ ]:
sc = preprocessing.StandardScaler()
iris_data2 = sc.fit(iris_data).transform(iris_data)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(iris_data2, iris_target)

In [ ]:
X_train2 = np.array([np.append(X_train[i], 1) for i in range(len(X_train))])
X_test2 = np.array([np.append(X_test[i], 1) for i in range(len(X_test))])

In [ ]:
feature_n = iris_data.shape[1]

In [ ]:
class_n = len(set(iris.target))

$\sigma(z)_{i} =  \dfrac{e^{z_{i}}}{\sum^{K}_{j=1}e^{z_{i}}} \ or \ \sigma(z)_{i} =  \dfrac{e^{-z_{i}}}{\sum^{K}_{j=1}e^{-z_{i}}} \ for \ i = 1, ..., K$

In [ ]:
def softmax(w, x, c):
  class_vector = np.array([[np.sum(np.exp(w[i] @ x.T))] for i in range(class_n)])
  class_vector = class_vector / np.sum(class_vector)
  return class_vector[c]

def grad_softmax(w, x, c):
  sm = softmax(w, x, c)
  return sm * (1 - sm) * x

$\dfrac{\partial}{\partial z_{i}}\sigma(z)_{i} = \dfrac{e^{z_{i}}}{\sum^{K}_{j=1}e^{z_{i}}} \ - \ e^{z_{i}} \cdot \dfrac{e^{z_{i}}}{\left(\sum^{K}_{j=1}e^{z_{i}}\right)^{2}} \ = \ \dfrac{e^{z_{i}}}{\sum^{K}_{j=1}e^{z_{i}}} \cdot \left(1-\dfrac{e^{z_{i}}}{\sum^{K}_{j=1}e^{z_{i}}}\right) \ = \ \sigma(z)_{i}\left(1-\sigma(z)_{i}\right)$

In [ ]:
X_class_list = [[], [], []]
for i in range(len(Y_train)):
  X_class_list[Y_train[i]].append(X_train2[i].tolist())

In [ ]:
epsilon = 0.001
w = np.array([np.array([5. for k in range(feature_n + 1)]) for c in range(class_n)])

In [ ]:
t1 = time.time()
for c in range(class_n):
  length = len(X_class_list[c])
  for i in range(10000):
    w[c] = w[c] + epsilon * grad_softmax(w, np.array(X_class_list[c]), c)[random.randint(0, length - 1)]
t2 = time.time()
print("total time : {}\n{}".format((t2 - t1), w))

In [ ]:
X_max, X_step = 4, 0.02
X_region = int(2*X_max/X_step)

In [ ]:
t1 = time.time()

X_axis = [-X_max + n*X_step for n in range(X_region)]
X_plane = np.array([[
                     [-X_max + n*X_step, -X_max + m*X_step, 1] for n in range(X_region)
                     ] for m in range(X_region)])
Z = np.array([[
               float(
                   np.argmax(
                       [softmax(w, X_plane[m][n], c) for c in range(class_n)]
                       )
                   ) for n in range(X_region)
                   ] for m in range(X_region)])

t2 = time.time()
print("total time : {}\n{}".format((t2 - t1), w))

In [ ]:
plt.figure(figsize=(8,8))
plt.contourf(X_axis, X_axis, Z, cmap='Wistia')
color = ['red', 'green', 'blue']
for i in range(class_n):
  X = np.array(X_class_list[i])
  plt.scatter(X[:, 0], X[:, 1], c=color[i])

plt.show()

## 29. 線形回帰（Linear regression）

### <font color = blue>**1.** </font> 単回帰

単回帰の前提
1. 直線(1次関数)で近似できる <--- **かなり強い仮定**
2. 二乗和誤差最小が最も良い
3. 誤差が独立である

#### <font color = green> **1.1.** </font> データの準備

In [ ]:
# ライブラリのインポート
from random import randint
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# 乱数を固定
np.random.seed(24)

# 表示する桁数の設定
np.set_printoptions(precision=4)

In [ ]:
## データの準備
# 10個のデータを生成する
# 真の分布は y = 2 + 3x + ε (ε: ノイズ)

N1 = 10

# 0.0以上、1.0未満の乱数
X1 = np.random.rand(N1, 1)

y1 = 2 + 3 * X1 + np.random.rand(N1, 1)

In [ ]:
# 可視化
plt.figure(figsize=(8,5))
plt.scatter(X1, y1)
plt.xlim(0.0, 1.2)
plt.ylim(0.0, 7.0)

plt.show()

#### <font color = green> **1.2.** </font> 1.1. のデータになるべくフィットするような直線を引く

In [ ]:
# ライブラリのインポート
from sklearn.linear_model import LinearRegression

In [ ]:
linreg1 = LinearRegression()

linreg1.fit(X1, y1.ravel())
# linreg1.fit(X1, y1.reshape(-1)) # これでもOK

In [ ]:
# 係数を取得
w0 = linreg1.intercept_
w1 = linreg1.coef_

# 小数点以下第２位まで表示させる場合
print("w0 = {:.2f},  w1 = {:.2f}".format(w0, w1[0]))

In [ ]:
#####################################
# 真の分布は 切片:2 傾き:3 だから、そこそこ良さそう

In [ ]:
# 最小二乗誤差
from sklearn.metrics import mean_squared_error as mse

mse(y1, linreg1.predict(X1))

In [ ]:
# 可視化
plt.plot([0, 1],
         w0 + w1 * [0, 1],
         'red')

plt.scatter(X1, y1)

#### <font color = green> **1.3.** </font> データを増やすと真の分布の再現性能が向上するか？

In [ ]:
# 200個のデータを生成する
# 真の分布は y = 2 + 3x + ε (ε: ノイズ)
N2 = 50

X2 = np.random.rand(N2, 1)
y2 = 2 + 3 * X2 + np.random.rand(N2, 1)

In [ ]:
# 可視化
plt.figure(figsize=(8,5))
plt.scatter(X2, y2)
plt.xlim(-0.2, 1.2)
plt.ylim(0.0, 7.0)

plt.show()

In [ ]:
linreg2 = LinearRegression()

linreg2.fit(X2, y2.ravel())

In [ ]:
# 係数を取得
w0_2 = linreg2.intercept_
w1_2 = linreg2.coef_

# 小数点以下第２位まで表示させる場合
print("w0 = {:.2f},  w1 = {:.2f}".format(w0_2, w1_2[0]))

In [ ]:
#####################
# 真の分布は 切片:2 傾き:3
#
# データ点10個での結果は
# w0 = 2.71, w1 = 2.77
# だった

In [ ]:
# 最小二乗誤差
mse(y2, linreg2.predict(X2))

In [ ]:
# 可視化
plt.plot([0, 1],
         w0_2 + w1_2 * [0, 1],
         'red')

plt.scatter(X2, y2)

### <font color = blue>**2.** </font> 単回帰では対応できない曲線

In [ ]:
## こんなデータの場合

N3 = 10
X3 = np.linspace(0, 1, N3) # 配列の状態
noise = np.random.uniform(low=-1.0, high=1.0, size=N3) * 0.35

# 真の分布: y = sin(2πx) + ε
y3 = np.sin(2.0 * np.pi * X3) + noise

# 可視化
plt.scatter(X3, y3)
plt.show()

#### <font color = green> **2.1.** </font> 真の分布を確認
（今は自分で生成した練習用データなので真の分布を知っている）

In [ ]:
Xs = np.linspace(0, 1, 100)
sinX = np.sin(2.0 * np.pi * Xs)
plt.plot(Xs, sinX, color='green')
plt.show()

In [ ]:
## 観測したデータと重ねて表示してみる

plt.scatter(X3, y3)
plt.plot(Xs, sinX, color='green')
plt.show()

#### <font color = green> **2.2.** </font> 単回帰でやってみる

In [ ]:
linreg3 = LinearRegression()

linreg3.fit(X3.reshape(-1, 1), y3) # 配列の状態のX3を縦ベクトルに変換

In [ ]:
w0_3 = linreg3.intercept_
w1_3 = linreg3.coef_[0]

print("w0 = {:.2f},  w1 = {:.2f}".format(w0_3, w1_3))

In [ ]:
# 可視化

plt.plot(Xs, w0_3 + w1_3 * Xs, color='red')
plt.scatter(X3, y3)
plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
# 最小二乗誤差
mse(y3, linreg3.predict(X3.reshape(-1, 1)))

In [ ]:
##################################
# 真の分布からは程遠い
# が、真の分布を知らない状態で見るとどうだろう？

In [ ]:
plt.plot(Xs, w0_3 + w1_3 * Xs, color='red')
plt.scatter(X3, y3)
# plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## 1次関数で当てはらないのであれば、2次関数や3次関数で近似すればいいのでは？

#### <font color = green> **2.3.** </font> 2次までの多項式でやってみる
- 使う変数(説明変数)は同じく $x$ のみ
- $y = w_0 + w_1 x + w_2 x^2$

In [ ]:
X3_2 = X3**2

In [ ]:
X4 = np.concatenate([X3.reshape(-1, 1), X3_2.reshape(-1, 1)], axis=1)
y4 = y3

X4.shape

In [ ]:
linreg4 = LinearRegression()

linreg4.fit(X4, y4)

In [ ]:
w0_4 = linreg4.intercept_
w1_4 = linreg4.coef_[0]
w2_4 = linreg4.coef_[1]

print("w0 = {:.2f},  w1 = {:.2f},  w2 = {:.2f}".format(w0_4, w1_4, w2_4))

In [ ]:
# 最小二乗誤差
mse(y4, linreg4.predict(X4))

In [ ]:
# 可視化
plt.plot(Xs, w0_4 + w1_4 * Xs + w2_4 * Xs**2, color='red')

plt.scatter(X3, y3)
# plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## 若干カーブした
## 単回帰よりよくなった？ とも思えない。複雑度が足りない

#### <font color = green> **2.4.** </font> 3次の項までとる
- 次数をあげることで複雑度を向上させる(≒カーブに対応させる)

- $y = w_0 + w_1 x + w_2 x^2 + w_3 x^3$

In [ ]:
## 多項式を準備してくれるライブラリ
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
# 今3次なので
poly = PolynomialFeatures(degree=3)

poly.fit(X3.reshape(-1, 1))

X5 = poly.transform(X3.reshape(-1, 1))
y5 = y3
X5

In [ ]:
linreg5 = LinearRegression().fit(X5, y5)

In [ ]:
Xs_5 = poly.fit_transform(Xs.reshape(-1, 1))

plt.plot(Xs, linreg5.predict(Xs_5), color='red')

plt.scatter(X3, y3)
plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
plt.plot(Xs, linreg5.predict(Xs_5), color='red')

plt.scatter(X3, y3)
# plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## だいぶいい感じ！

#### <font color = green> **2.5.** </font> ほかの次数も一気に試す
- さらに次数を増やせばもっと当てはまりがよくなるのでは？

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip([3, 5, 7, 50], axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X3.reshape(-1, 1))
  linreg = LinearRegression().fit(X_poly, y3)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, linreg.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X3, y3)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
#############################
# 次数を大きくしすぎると過学習がおきる
# - 全ての点を通過するようになる
# - 係数が大きくなりすぎる

In [ ]:
#### 変数を変えて遊ぶ ####

# このlistの数字を変更してセルを実行 #
Dim = [10, 20, 30, 40]


fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip(Dim, axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X3.reshape(-1, 1))
  linreg = LinearRegression().fit(X_poly, y3)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, linreg.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X3, y3)
  ax.plot(Xs, sinX, color='green')

plt.show()

### <font color = blue>**3.** </font> 過学習の対策その1 : データを増やす

In [ ]:
## 2. の正弦波データについて、データ点を 10個 -> 100個 としてみる

In [ ]:
N = 100

# start から stop の範囲でデータを N個 つくる
X = np.linspace(start=0, stop=1, num=N)
# X = np.linspace(start=0.5, stop=0.7, num=N) # データの生成範囲を狭めると...?

noise = np.random.uniform(low=-1.0, high=1.0, size=N) * 0.3

# 真の分布: y = sin(2πx) + ε
y = np.sin(2.0 * np.pi * X) + noise

In [ ]:
# 可視化
plt.scatter(X, y)
plt.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
Dim = [5, 7, 9, 30]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip(Dim, axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  linear_regression = LinearRegression().fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, linear_regression.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## 次数が大きくなったときの過学習が抑制されているのが分かる

In [ ]:
#### 変数を変えて遊ぶ ####

# このlistの数字を変更してセルを実行 #
Dim = [5, 7, 9, 30]


fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip(Dim, axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  linear_regression = LinearRegression().fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, linear_regression.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

### <font color = blue>**4.** </font> 過学習の対策その2 : 正則化

In [ ]:
# 再びデータ数を10にする
N = 10

X = np.linspace(0, 1, N)
noise = np.random.uniform(low=-1.0, high=1.0, size=N) * 0.3

# 真の分布: y = sin(2πx) + ε
y = np.sin(2.0 * np.pi * X) + noise

In [ ]:
# 観測したデータと重ねてみる
plt.scatter(X, y)
plt.plot(Xs, sinX, color='green')

plt.show()

#### <font color = green> **4.1.** </font> リッジ回帰

$\underset{w}{min}\left(\|X_{w} - y\|^{2}_{2} + \alpha \|w\|^{2}_{2}\right)$

In [ ]:
# ライブラリのインポート
from sklearn.linear_model import Ridge

In [ ]:
Dim = [5, 7, 9, 100]

Alpha = 1.0
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip(Dim, axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  
  # alpha によって正則化の度合いを制御する
  ridge = Ridge(alpha=Alpha).fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, ridge.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## alphaの違いを見てみる

# 今、多項式の次数は11次で固定
Dim = 11

Alpha = [0.001, 0.01, 0.1, 1.0]


fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for alpha, ax in zip(Alpha, axes):
  poly = PolynomialFeatures(degree=11)
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  
  ridge = Ridge(alpha=alpha).fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, ridge.predict(Xs_poly), color='red')
  ax.set_title("alpha = {}".format(alpha))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
#### 変数を変えて遊ぶ ####

# このlistの数字を変更してセルを実行 #
Dim = [5, 7, 9, 100]
Alpha = [0.001, 0.01, 0.1, 1.0]




fig, axes = plt.subplots(4, 4, figsize=(16, 16))
for i in range(len(Dim)):
  poly = PolynomialFeatures(degree=Dim[i])
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))
  for k in range(len(Alpha)):
    ridge = Ridge(alpha=Alpha[k]).fit(X_poly, y)
    axes[i][k].plot(Xs, ridge.predict(Xs_poly), color='red')
    axes[i][k].set_title("M = {}, alpha = {}".format(Dim[i], Alpha[k]))
    axes[i][k].scatter(X, y)
    axes[i][k].plot(Xs, sinX, color='green')
plt.show()

#### <font color = green> **4.2.** </font> ラッソ回帰

$\underset{w}{min}\left(\dfrac{1}{2n_{samples}}\|X_{w} - y\|^{2}_{2} + \alpha \|w\|_{1}\right)$

In [ ]:
# ライブラリのインポート
from sklearn.linear_model import Lasso

In [ ]:
Dim = [5, 7, 9, 11]

Alpha = 1.0

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for degree, ax in zip(Dim, axes):
  poly = PolynomialFeatures(degree)
  X_poly = poly.fit_transform(X.reshape(-1, 1))

  # alpha によって正則化の度合いを制御する
  lasso = Lasso(alpha=Alpha).fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, lasso.predict(Xs_poly), color='red')
  ax.set_title("M = {}".format(degree))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
## alphaの違いを見てみる

# 今、多項式の次数は7次で固定
Dim = 7

Alpha = [0.01, 0.05, 0.1, 1]

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

for alpha, ax in zip(Alpha, axes):
  poly = PolynomialFeatures(degree=Dim)
  X_poly = poly.fit_transform(X.reshape(-1, 1))

  lasso = Lasso(alpha=alpha).fit(X_poly, y)
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))

  ax.plot(Xs, lasso.predict(Xs_poly), color='red')
  ax.set_title("alpha = {}".format(alpha))
  ax.scatter(X, y)
  ax.plot(Xs, sinX, color='green')

plt.show()

In [ ]:
#### 変数を変えて遊ぶ ####

# このlistの数字を変更してセルを実行 #
Dim = [5, 7, 9, 11]
Alpha = [0.01, 0.025, 0.05, 0.1]




fig, axes = plt.subplots(4, 4, figsize=(16, 16))
for i in range(len(Dim)):
  poly = PolynomialFeatures(degree=Dim[i])
  X_poly = poly.fit_transform(X.reshape(-1, 1))
  Xs_poly = poly.fit_transform(Xs.reshape(-1, 1))
  for k in range(len(Alpha)):
    lasso = Lasso(alpha=Alpha[k]).fit(X_poly, y)
    axes[i][k].plot(Xs, lasso.predict(Xs_poly), color='red')
    axes[i][k].set_title("M = {}, alpha = {}".format(Dim[i], Alpha[k]))
    axes[i][k].scatter(X, y)
    axes[i][k].plot(Xs, sinX, color='green')
plt.show()

#### <font color = green> **4.3.** </font> リッジ回帰 と ラッソ回帰 の係数を比較

In [ ]:
# 多項式は7次で統一
Dim = 7

poly = PolynomialFeatures(degree=Dim)
X_poly = poly.fit_transform(X.reshape(-1, 1))

# alpha = 0.01 で統一
Alpha = 0.01

ridge = Ridge(alpha=Alpha).fit(X_poly, y)
lasso = Lasso(alpha=Alpha).fit(X_poly, y)

In [ ]:
# 両者の係数を比較する
### 表示桁と位置を揃えるために変なことしています ###
str1 = "Ridge : [{:.02f}".format(ridge.coef_[0])
for i in range(1, len(ridge.coef_)):
  str1 += ", {:.02f}".format(ridge.coef_[i])
str1 += "]"

str2 = "Lasso : [{:.02f}".format(lasso.coef_[0])
for i in range(1, len(lasso.coef_)):
  str2 += ", {:.02f}".format(lasso.coef_[i])
str2 += "]"


print(str1)
print(str2)

#### <font color = green> **4.4.** </font> リッジ回帰、ラッソ回帰、それらを組み合わせたエラスティックネットの視覚イメージ

sklearn.linear_model.SGDClassifier 公式サンプルコードより

In [ ]:
## SGD: Penalties
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
l1_color = "navy"
l2_color = "c"
elastic_net_color = "darkorange"

line = np.linspace(-1.5, 1.5, 1001)
xx, yy = np.meshgrid(line, line)

l2 = xx ** 2 + yy ** 2
l1 = np.abs(xx) + np.abs(yy)

rho = 0.5 ###
elastic_net = rho * l1 + (1 - rho) * l2

In [ ]:
plt.figure(figsize=(7, 7), dpi=100)
ax = plt.gca()

elastic_net_contour = plt.contour(xx, yy, elastic_net, levels=[1],
                                  colors=elastic_net_color)
l2_contour = plt.contour(xx, yy, l2, levels=[1], colors=l2_color)
l1_contour = plt.contour(xx, yy, l1, levels=[1], colors=l1_color)
ax.set_aspect("equal")
ax.spines['left'].set_position('center')
ax.spines['right'].set_color('none')
ax.spines['bottom'].set_position('center')
ax.spines['top'].set_color('none')

plt.clabel(elastic_net_contour, inline=1, fontsize=18,
           fmt={1.0: 'elastic-net'}, manual=[(-1, -1)])
plt.clabel(l2_contour, inline=1, fontsize=18,
           fmt={1.0: 'L2'}, manual=[(-1, -1)])
plt.clabel(l1_contour, inline=1, fontsize=18,
           fmt={1.0: 'L1'}, manual=[(-1, -1)])

plt.tight_layout()
plt.show()